# Using EAGO's basic optimizer with user-defined subroutines 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

### Overview  
In this section, we construct an optimizer that uses EAGO's basic nlp solution routine with user-defined lower and upper bounding problems. The **EAGO.Optimizer** structure supplies a number of parameters and stored structures that advanced users may find useful for constructing specialized solution routines. For a full review, of the EAGO.optimizer object the reader is directed to the **EAGO.Optimizer** docstring and documentation provided at [https://psorlab.github.io/EAGO.jl/stable/](https://psorlab.github.io/EAGO.jl/stable/).

In this example, we'll forgo extensive integration into the EAGO.optimizer and simply replace the lower and upper-bounding problems to construct B&B routine that solves the following problem to global optimality using bounds obtained from interval arithmetic:

$
\begin{align}
&\min_{\mathbf x \in X} \;\; \sin(x_1)x_2^2 - \cos(x_3) / x_4 \label{eq1} \\
&X = [-10,10]\times[-1,1]\times[-10,10]\times[2,20].
\end{align}
$

We begin importing EAGO, IntervalArithmetic[1], and JuMP[2].

In [1]:
using EAGO, IntervalArithmetic, JuMP

We now define the IntervalExt struct as a subtype of the EAGO.Extension Type.

In [2]:
struct IntervalExt <: EAGO.ExtensionType end

### Defining a custom lower bounding problem
A valid lower bound is obtained from the lower bound of the natural interval extension using the **ValidatedNumerics.jl** package. The LowerProblem used accepts the **EAGO.Optimizer** structure and a **EAGO.NodeBB** structure, computes the bound by method overloading interval arithmetic, and stores the results to the appropriate field of the **EAGO.Optimizer's**. Note that the problem is unconstrained on the domain so we can assume it is always feasible. Further since the interval bound is constrain along the entire domain associated with a node no additional cuts will be beneficial and thus we've disabled them using the `_cut_add_flag` field.

In [7]:
import EAGO.lower_problem!
function lower_problem!(t::IntervalExt, x::EAGO.Optimizer)
    
    # retrieve bounds at current node
    n = x._current_node
    lower = n.lower_variable_bounds
    upper = n.upper_variable_bounds
    
    # define X for node and compute interval extension
    x_value = Interval.(lower, upper)
    F = sin(x_value[1])x_value[2]^2-cos(x_value[3])/x_value[4]
    
    x._lower_objective_value = F.lo
    x._lower_solution = mid.(x_value)
    x._lower_feasibility = true
    x._cut_add_flag = false
    
    return
end

lower_problem! (generic function with 3 methods)

### Defining a custom upper bounding problem
Since the problem is unconstained, any feasible point represents a valid upper bound. If we arbitrarily take evaluate the function at the midpoint we obtain an upper bound. The below function constructs an upper bound in this manner then stores the results to the appropriate field of the **EAGO.Optimizer's**.

In [4]:
import EAGO.upper_problem!
function EAGO.upper_problem!(t::IntervalExt, x::EAGO.Optimizer)
    
    # retrieve bounds at current node
    n = x._current_node
    lower = n.lower_variable_bounds
    upper = n.upper_variable_bounds
    
    # compute midpoint value
    x_value = 0.5*(upper + lower)
    f_val = sin(x_value[1])*x_value[2]^2-cos(x_value[3])/x_value[4]
    x._upper_objective_value = f_val
    x._upper_solution = x_value
    x._upper_feasibility = true
    
    return
end

upper_problem! (generic function with 1 method)

### Disable unecessary routines.
It is entirely possible to disable domain reduction by manipulating keyword arguments supplied to the optimizer. However, for simplicity sake we'll simply overload the default preprocessing and postprocessing methods,

In [5]:
import EAGO: preprocess!, postprocess!, cut_condition
function EAGO.preprocess!(t::IntervalExt, x::Optimizer)
    x._preprocess_feasibility = true
    return
end
function EAGO.postprocess!(t::IntervalExt, x::Optimizer)
    x._postprocess_feasibility = true
    return
end
EAGO.cut_condition(t::IntervalExt, x::Optimizer) = false

### Build the JuMP Model and optimize
We now add our optimizer to a JuMP model, provide variable bounds, optimize. Note that options can be provided to the EAGO optimizer using a series of keywords of a Dict{Symbol,Any} object. Both manners of providing options to EAGO are illustrated below.

In [ ]:
# Creates a JuMP model with the the lower_problem, upper_problem, and absolute tolerance set by keyword arguments
m = JuMP.Model(with_optimizer(EAGO.Optimizer, absolute_tolerance = 0.001, obbt_depth = 0, dbbt_depth = 0, cp_depth = 0,
                              branch_variable = Bool[true; true; true; true], ext_type = IntervalExt()))

# Create the same model m using an options dictionary
opt_dict = Dict{Symbol, Any}()
opt_dict[:absolute_tolerance] = 0.001
opt_dict[:obbt_depth] = 0
opt_dict[:dbbt_depth] = 0
opt_dict[:cp_depth] = 0
opt_dict[:branch_variable] = Bool[true; true; true; true]
opt_dict[:ext_type] = IntervalExt()

m = JuMP.Model(with_optimizer(EAGO.Optimizer; opt_dict...))

# Adds variables and bounds
x_L = [-10, -1, -10, 2]
x_U = [10, 1, 10, 20]
@variable(m, x_L[i] <= x[i=1:4] <= x_U[i])

# Solves the problem
JuMP.optimize!(m)

### Get information from the JuMP Model object
The objective value, solution, termination status,  and primal status can then be accessed via the standard JuMP interface.

In [6]:
fval = JuMP.objective_value(m)
xsol = JuMP.value.(x)
status_term = JuMP.termination_status(m)
status_prim = JuMP.primal_status(m)

println("EAGO terminated with a status of $status_term and a result code of $status_prim")
println("The optimal value is: $fval, the solution found is $xsol.")

EAGO terminated with a status of OPTIMAL and a result code of FEASIBLE_POINT
The optimal value is: -1.691376119161244, the solution found is [0.3125, 0.03125, 0.3125, 0.5625].


### Advice for more advanced constructions
The *default_lower_problem* and *default_upper_problem* should be used templates for error handling and retreiving information from MOI models. Similarly, the other default routine are good starting points for building custom modifications.

Essentially all of EAGO's subroutines are stored to a field in the **EAGO.Optimizer** structure can be reset as user-defined functions.

### References
1. IntervalArithmetic.jl [Computer software] (2019). Retrieved from https://github.com/JuliaIntervals/IntervalArithmetic.jl
2. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, **SIAM** 59 (2017), pp. 295-320.